In [ ]:
import os
import random
import collections
import operator
import itertools
import pickle
import tqdm
import pprint

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import simpleclock
import sklearn.metrics
import sklearn.decomposition

In [ ]:
import torch
import torchtext

## Dataset, iterators

In [ ]:
data_path = os.path.join(os.path.abspath(''), "data_cine_cleaned.csv")

In [ ]:
TEXT = torchtext.data.Field(tokenize = "spacy",
                            tokenizer_language="fr_core_news_sm",
                            include_lengths=True)
LABEL = torchtext.data.LabelField(dtype=torch.float, use_vocab=False, preprocessing=lambda x: float(x) / 5)
# labels are linearly rescaled to a 0-1 range
# todo: test if preprocessing data before isn't faster

In [ ]:
dataset = torchtext.data.TabularDataset(path=data_path,
                                        format="CSV",
                                        fields={"critique": ("input", TEXT), "note": ("target", LABEL)})

In [ ]:
data_train, data_test = dataset.split(split_ratio=0.8)

In [ ]:
data_train, data_valid = data_train.split(split_ratio=0.8)

In [ ]:
print(f"""training data: {len(data_train)} examples.
validation data: {len(data_valid)} examples.
test data: {len(data_test)} examples.""")

In [ ]:
vectors = torchtext.vocab.Vectors("cc.fr.300.vec", os.path.join(os.path.expanduser("~"), "Downloads"))

In [ ]:
VOCAB_MAX_SIZE = 70000
TEXT.build_vocab(data_train, max_size=VOCAB_MAX_SIZE, vectors=vectors)

In [ ]:
DEVICE = torch.device("cuda")

In [ ]:
BATCH_SIZE = 256

In [ ]:
iter_train, iter_valid, iter_test = \
    torchtext.data.BucketIterator.splits(datasets=(data_train, data_valid, data_test),
                                         batch_size=BATCH_SIZE,
                                         device=DEVICE,
                                         sort_within_batch=True,
                                         sort_key=lambda example: len(example.input),
                                         sort=False)

## Model definition

In [ ]:
class RNN(torch.nn.Module):
    def __init__(self, n_vocab, embedding_dim, hidden_dim, output_dim, dropout, bidirectional,
                 n_layers, pad_idx):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.bidirectional = bidirectional
        num_dir = 2 if bidirectional else 1
        self.embedding = torch.nn.Embedding(n_vocab, embedding_dim, padding_idx=pad_idx)
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim,
                                 bidirectional=bidirectional,
                                 num_layers=n_layers,
                                 dropout=dropout)
        self.fc = torch.nn.Linear(hidden_dim * num_dir, output_dim)
        self.dropout = torch.nn.Dropout(dropout)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, input_lengths):
        input, lengths = input_lengths
        torch.nn.utils.rnn.pack_padded_sequence(input, lengths)
        embedded = self.embedding(input)  # ((sent_len, batch), emb_dim)
        packed_output, (hidden, cell) = self.rnn(embedded)  # hidden: (num_layers * num_directions,
                                                            #          batch, hidden_size * num_directions)
        hidden = (torch.cat([hidden[-2, :, :], hidden[-1, :, :]], dim=1)
                  if self.bidirectional else hidden).squeeze(0)  # (batch, hidden_size * num_directions)
        return self.sigmoid(self.fc(self.dropout(hidden)))  # (batch, 1)

In [ ]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
N_VOCAB = len(TEXT.vocab)

In [ ]:
DEFAULT_PARAMS = {
    "n_vocab": N_VOCAB,
    "embedding_dim": 300,
    "hidden_dim": 256,
    "output_dim": 1,
    "dropout": 0.5,
    "bidirectional": True,
    "n_layers": 1,
    "pad_idx": PAD_IDX,
}

def default_model(**kwargs):
    _d = {}
    _d.update(DEFAULT_PARAMS)
    _d.update(kwargs)
    return RNN(**_d)

In [ ]:
def pseudo_init(model, criterion, device=DEVICE, learn_embedding_param=True):
    model.embedding.weight.data.copy_(TEXT.vocab.vectors)
    model.embedding.weight.data[UNK_IDX] = torch.zeros(model.embedding_dim)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(model.embedding_dim)
    
    for name, param in model.named_parameters():
        if name == "embedding.weight":
            param.requires_grad = learn_embedding_param
    
    print("The model has {:,} trainable parameters"
         .format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    return model, criterion

In [ ]:
def output_to_pred(output):
    return (output * 10).round() / 2

## Saving/loading utils

In [ ]:
def save_vocab_embedding(path, vocab, embedding):
    with open(path, "w") as f:
        for word, vector in tqdm.tqdm(zip(vocab.itos, embedding)):
            
            # skip words with unicode symbols
            if len(word) != len(word.encode()):
                continue
            
            # 'words' like " " or "\n" fail to be loaded
            if word.strip() == "":
                continue

            f.write(f"{word} {' '.join(str(e) for e in vector.tolist())}\n")

In [ ]:
def load_context(model, field, path_model, path_vocab, cache_embeddings="cache_embeddings",
                 batch_size=BATCH_SIZE, device=DEVICE):
    
    _vectors = torchtext.vocab.Vectors(path_vocab, cache_embeddings)  # voir unk_init
    field.build_vocab(data_train, max_size=VOCAB_MAX_SIZE, vectors=_vectors)
    
    model.load_state_dict(torch.load(path_model))
    
    model.embedding.weight.data.copy_(field.vocab.vectors)
    
    model = model.to(device)
    
    iter_train, iter_valid, iter_test = \
        torchtext.data.BucketIterator.splits(datasets=(data_train, data_valid, data_test),
                                             batch_size=batch_size,
                                             device=device,
                                             sort_within_batch=True,
                                             sort_key=lambda example: len(example.input),
                                             sort=False)
    
    return model, (iter_train, iter_valid, iter_test)

In [ ]:
def save_itos(itos, path):
    with open(path, "wb") as f:
        pickle.dump(itos, f)

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
def save_for_pred(folder_path, model=None, itos=None):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    if model is not None:
        save_model(model, os.path.join(folder_path, "model.pt"))
    if itos is not None:
        save_itos(itos, os.path.join(folder_path, "itos.pickle"))

In [ ]:
def load_for_pred(folder_path, model, device=DEVICE):
    model.load_state_dict(torch.load(os.path.join(folder_path, "model.pt")))
    model = model.to(DEVICE)
    with open(os.path.join(folder_path, "itos.pickle"), "rb") as f:
        itos = pickle.load(f)
    stoi = collections.defaultdict(lambda: 0)
    stoi.update(map(lambda t: t[::-1], enumerate(itos)))
    return model, itos, stoi

## Training utils

In [ ]:
def accuracy(preds, y):
    correct = (preds == y).float()  # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        output = model(batch.input).squeeze(1)
        loss = criterion(output, batch.target)
        loss.backward()
        optimizer.step()
        
        acc = accuracy(output_to_pred(output), batch.target * 5)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
      
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            output = model(batch.input).squeeze(1)
            loss = criterion(output, batch.target)
            acc = accuracy(output_to_pred(output), batch.target * 5)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# will break easily...
class MinMaxList(list):
    def __init__(self):
        super().__init__()
        self._min = float("inf")
        self._max = -float("inf")
    
    @property
    def min(self):
        return self._min
    
    @property
    def max(self):
        return self._max
    
    def append(self, e):
        super().append(e)
        if e > self._max:
            self._max = e
        if e < self._min:
            self._min = e

In [ ]:
class TrainInfo:
    def __init__(self, valid={}, train={}):
        self.valid = collections.defaultdict(MinMaxList)
        self.valid.update(valid)
        self.train = collections.defaultdict(MinMaxList)
        self.train.update(train)
    
    def save(self, path):
        packed = {
            "valid": dict(self.valid),
            "train": dict(self.train),
        }
        with open(path, "wb") as f:
            pickle.dump(packed, f)
    
    @classmethod
    def load(cls, path):
        with open(path, "rb") as f:
            packed = pickle.load(f)
            return cls(valid=packed["valid"],
                       train=packed["train"])
    
    @staticmethod
    def _dict_to_repr(d):
        return dict(map(lambda k_v: (k_v[0], f"{len(k_v[1])} elements"), d.items()))
    
    def __repr__(self):
        return pprint.pformat({"valid": self._dict_to_repr(self.valid),
                     "train": self._dict_to_repr(self.train),})

In [ ]:
def do_training(model, name, iter_train, iter_valid, optimizer, criterion, fun_train,
                fun_eval, n_epochs=100, training_info=None, fun_stop=lambda: False, fun_save=None):
    
    if fun_save is None:
        print("Not saving best model.")
    
    clock = simpleclock.Clock.started()
    torch.cuda.empty_cache()
    train_info = training_info if training_info is not None else TrainInfo()

    print(f"Starting training: {n_epochs} epochs.")
    for epoch in range(n_epochs):
        if fun_stop():
            print("Interrupted. (fun_stop)")
            break

        clock.elapsed_since_start.call()  # meh

        train_loss, train_acc = fun_train(model, iter_train, optimizer, criterion)
        valid_loss, valid_acc = fun_eval(model, iter_valid, criterion)
        
#         is_best = valid_loss < train_info.valid["loss"].min
        is_best = valid_acc > train_info.valid["acc"].max
        print("Epoch: {e:.<{max_len}}. T, V acc: {train:.1f}%, {valid:.1f}%. Took {t:.2f}s."
             .format(e=epoch + 1,
                     max_len=len(str(n_epochs)),
                     train=100 * train_acc,
                     valid=100 * valid_acc,
                     t=clock.elapsed_since_last_call())
             + (" (+)" if is_best else "")
             )
        
        train_info.train["loss"].append(train_loss)
        train_info.valid["loss"].append(valid_loss)
        train_info.train["acc"].append(train_acc)
        train_info.valid["acc"].append(valid_acc)

        if is_best:
            if fun_save is not None:
                fun_save(model)

    clock.elapsed_since_start.print(f"Trained {name}, {n_epochs} epochs, for")
    return train_info

In [ ]:
class TrainSet:
    def __init__(self, model, name, iter_train, iter_valid,
                 fun_optimizer, fun_criterion, fun_train, fun_eval,
                 device=DEVICE, n_epochs=100, training_info=None, fun_stop=lambda: False):
        self.model = model
        self.name = name
        self.iter_train = iter_train
        self.iter_valid = iter_valid
        self.fun_optimizer = fun_optimizer
        self.fun_criterion = fun_criterion
        self.fun_train = fun_train
        self.fun_eval = fun_eval
        self.n_epochs = n_epochs
        self.device = device
        self.training_info = training_info
        self.fun_stop = fun_stop
        
        self.optimizer = None
        self.criterion = None
        
    def init(self, learn_embedding_param=True):
        self.model, self.criterion = pseudo_init(self.model, self.fun_criterion(), self.device,
                                                 learn_embedding_param=learn_embedding_param)
        self.optimizer = self.fun_optimizer(self.model.parameters())
    
    def do_training(self, fun_save=None):
        if self.optimizer is None or self.criterion is None:
            raise Exception("It looks like an init is needed: optimizer or criterion is None")
        return do_training(model=self.model,
                           name=self.name,
                           iter_train=self.iter_train,
                           iter_valid=self.iter_valid,
                           optimizer=self.optimizer,
                           criterion=self.criterion,
                           fun_train=self.fun_train,
                           fun_eval=self.fun_eval,
                           n_epochs=self.n_epochs,
                           training_info=self.training_info,
                           fun_stop=self.fun_stop,
                           fun_save=fun_save)

## Training

In [ ]:
train_sets = []
N_EPOCHS = 3

for hidden_dim, n_layers in itertools.product([16], [3]):
    train_sets.append(TrainSet(
        model=default_model(hidden_dim=hidden_dim, n_layers=n_layers),
        name=f"rnn_hidden-{hidden_dim}_nlayers-{n_layers}_nepochs-{N_EPOCHS}",
        iter_train=iter_train,
        iter_valid=iter_valid,
        fun_optimizer=torch.optim.Adam,
        fun_criterion=torch.nn.MSELoss,
        fun_train=train,
        fun_eval=evaluate,
        n_epochs=N_EPOCHS,
        fun_stop=lambda: os.path.exists(".stop"),
    ))

In [ ]:
for train_set in train_sets:
    folder_path = os.path.join(os.path.abspath(""), f"dump_{train_set.name}")
    
    train_set.init()
    training_info = train_set.do_training()
    
    # save itos, training_info. model is saved during training
    save_for_pred(folder_path, itos=TEXT.vocab.itos)
    training_info.save(os.path.join(folder_path, f"training_info.pickle"))
    
    # plot loss data ## todo: handle different data lengths?
    fig, axs = plt.subplots(2, 1, figsize=(20, 16))
    _n = len(training_info.train["loss"])
    ax_train = axs[0].plot(list(range(_n)), training_info.train["loss"], label="train")
    ax_valid = axs[0].plot(list(range(_n)), training_info.valid["loss"], label="valid")
    axs[0].set_title("Loss")
    ax_train = axs[1].plot(list(range(_n)), training_info.train["acc"], label="train")
    ax_valid = axs[1].plot(list(range(_n)), training_info.valid["acc"], label="valid")
    handles, labels = axs[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center')
    axs[1].set_title("Accuracy")
    plt.show()

## Peeking

In [ ]:
import spacy

nlp = spacy.load('fr_core_news_sm')

In [ ]:
def predict_tokens(tokens, model, stoi, device=DEVICE):
    model.eval()
    idxs = [stoi[t] for t in tokens]
    inp = torch.LongTensor(idxs).reshape(-1, 1).to(device)
    output = output_to_pred(model((inp, torch.LongTensor([len(tokens)]))))
    return output.item()


def predict(sentence, model, stoi=TEXT.vocab.stoi, tokenizer=nlp.tokenizer):
    return predict_tokens(list(map(str, tokenizer(sentence))), model, stoi)

class Predictor:
    def __init__(self, model, stoi=TEXT.vocab.stoi, tokenizer=nlp.tokenizer, device=DEVICE):
        self.model = model
        self.stoi = stoi
        self.tokenizer = tokenizer
        self.device = device
    
    def predict(self, sentence):
        return predict(sentence, self.model, self.stoi, self.tokenizer)
    
    def predict_tokens(self, tokens):
        return predict_tokens(tokens, self.model, self.stoi, self.device)
    
    def __call__(self, sentence):
        return self.predict(sentence)

In [ ]:
model, itos, stoi = load_for_pred("dump_rnn_hidden-128_nlayers-4_nepochs-1000",
                                  default_model(n_vocab=57051,
                                                hidden_dim=128,
                                                n_layers=4))

In [ ]:
# model = train_set.model

In [ ]:
predictor = Predictor(model, stoi=stoi)

In [ ]:
predictor("Un mauvais film."), \
predictor("Un bon film.")

In [ ]:
for example in random.sample(list(data_train), 1000):
    tokens, note = example.input, float(example.target)
    if note * 5 != predictor.predict_tokens(tokens):
        print(" ".join(tokens))
        print(f"true) {note * 5} - {predictor.predict_tokens(tokens)} (pred")

### Learned word vectors

In [ ]:
X = model.embedding.weight.data.clone().cpu()

In [ ]:
X

In [ ]:
def keep_word(word):
    return 4 < len(word) < 16

N_WORDS = 400

word_indexes = [i for i, word in enumerate(itos) if keep_word(word)]
word_indexes = random.sample(word_indexes, min(N_WORDS, len(word_indexes)))

In [ ]:
pca = sklearn.decomposition.PCA(n_components=2)

In [ ]:
Y = X[word_indexes]

In [ ]:
pca.fit(Y)
Z = pca.transform(Y)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
ax.scatter(Z[:, 0], Z[:, 1], color="white")
for i, row in zip(word_indexes, Z):
    ax.text(*row, itos[i], horizontalalignment="center", alpha=.75)
plt.show()

## Looser accuracies for evaluation

Is the model really doing bad if it predicts a 4.5 instead of a 5 ? There are at least two ways to allow for forgivable divergence with the test data :
* decrease notation's granularity, e.g. tranform the marks into good/bad, or good/bad/neutral.
* consider a prediction correct if it belongs to a 'small' interval containing the true value.

### Good/Neutral/Bad prediction

In [ ]:
def normalized_to_3_way(pred_tensor, bad_treshold=.375, good_treshold=.625):
    """np array with values: 0: bad, 1: neutral, 2: good"""
    return np.digitize(pred_tensor.cpu().detach().numpy(), [bad_treshold, good_treshold])
    
def eval_accuracy_3w(model, iterator):
    n_examples = 0
    n_success = 0
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        n_examples += len(batch)
        n_success += sum(normalized_to_3_way(predictions) == normalized_to_3_way(batch.target))
    return n_success / n_examples

def classif_report_3w(model, iterator):
    def preds_and_trues_to_array(predictions, true_targets):
        return np.concatenate([normalized_to_3_way(true_targets).reshape(-1, 1),
                               normalized_to_3_way(predictions).reshape(-1, 1)], axis=1)
    
    array = None
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        
        if array is None:
            array = preds_and_trues_to_array(predictions, batch.target)
        else:
            array = np.concatenate([array,
                                    preds_and_trues_to_array(predictions, batch.target)], axis=0)
    print(sklearn.metrics.classification_report(array[:, 0],
                                                array[:, 1],
                                                labels=[0, 1, 2],
                                                target_names=["bad", "neutral", "good"]))

In [ ]:
iterator = iter_test

classif_report_3w(model, iterator)


### Fuzzy accuracy

In [ ]:
def eval_accuracy_fuzzy(model, iterator, fuzziness=.1):
    n_examples = 0.
    n_success = 0.
    for batch in iterator:
        predictions = model(batch.input).squeeze(1)
        n_examples += len(batch)
        n_success += sum(torch.abs(predictions - batch.target) <= fuzziness).item()
    return n_success / n_examples

In [ ]:
eval_accuracy_fuzzy(model, iter_test, fuzziness=.21)

In [ ]:
xs = np.linspace(0, .5, 50)

In [ ]:
accs = [eval_accuracy_fuzzy(model, iter_test, fuzziness=x) for x in xs]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
ax.plot(xs, accs)

## Vocab differences

In [ ]:
unks = sents = 0
for batch in iter_train:
    sentences = batch.input[0]
    unks += (sentences == 0).sum().item()
    sents += batch.batch_size
print(f"avg number of <unk> per sentence: {unks / sents}")

In [ ]:
print(itos[10000:10010])
print(TEXT.vocab.itos[10000:10010])

In [ ]:
diff = set(itos).symmetric_difference(TEXT.vocab.itos)

In [ ]:
len(diff)